In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

In [2]:
# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of LSGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

go_gatesets = [ pygsti.optimize_gauge(gs, "target", targetGateset=gs_target, gateWeight=1, spamWeight=0.001) for gs in gs_lsgst_list]

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache

--- Iterative LSGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623488)

--- Iterative LSGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)

--- Iterative LSGST: Beginning iter 3 of 10 : 168 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 130.576 (168 data params - 40 model params = expected mean of 128; p-value = 0.420138)

--- Iterative LSGST: Beginning iter 4 of 10 : 441 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 422.579 (441 data params - 40 model params = expected mean of 401; p-value = 0.220005)

--- Iterative LSGST: Beginning iter 5 of 10 : 817 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 768.626 (817 data params - 40 model 

In [3]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    go_gatesets, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False)

In [4]:
#Make smaller reports (no confidence intervals or appendicies)
res.create_full_report_pdf(filename="tutorial_files/Example_reportB.pdf", verbosity=2, confidenceLevel=None,
                        debugAidsAppendix=False, gaugeOptAppendix=False,
                        pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_brief_report_pdf(filename="tutorial_files/Example_briefB.pdf", verbosity=2, confidenceLevel=None)

res.create_presentation_pdf(filename="tutorial_files/Example_slidesB.pdf", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_presentation_ppt(filename="tutorial_files/Example_slidesB.ppt", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

*** Generating tables ***
Generating targetSpamTable table...
Generating targetGatesTable table...
Generating datasetOverviewTable table...
Generating bestGatesetSpamTable table...
Generating bestGatesetSpamParametersTable table...
Generating bestGatesetGatesTable table...
Generating bestGatesetChoiTable table...
Generating bestGatesetDecompTable table...
Generating bestGatesetRotnAxisTable table...
Generating bestGatesetClosestUnitaryTable table...
Generating bestGatesetVsTargetTable table...
Generating bestGatesetErrorGenTable table...
Generating fiducialListTable table...
Generating rhoStrListTable table...
Generating EStrListTable table...
Generating germListTable table...
Generating chi2ProgressTable table...
*** Generating plots ***
 -- Chi2 plots (2):  1  Generating bestEstimateColorBoxPlot figure...
2  Generating invertedBestEstimateColorBoxPlot figure...

*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial ou

In [5]:
#Make large reports (with appendices and confidence regions)
res.create_full_report_pdf(filename="tutorial_files/Example_report.pdf", verbosity=2, confidenceLevel=95,
                        debugAidsAppendix=True, gaugeOptAppendix=True,
                        pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_brief_report_pdf(filename="tutorial_files/Example_brief.pdf", verbosity=2, confidenceLevel=95)

res.create_presentation_pdf(filename="tutorial_files/Example_slides.pdf", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_presentation_ppt(filename="tutorial_files/Example_slides.ppt", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

*** Generating tables ***
Generating targetSpamTable table...
Generating targetGatesTable table...
Generating datasetOverviewTable table...
Generating bestGatesetSpamTable table...
Generating bestGatesetSpamParametersTable table...
Generating bestGatesetGatesTable table...
Generating bestGatesetChoiTable table...
Generating bestGatesetDecompTable table...
Generating bestGatesetRotnAxisTable table...
Generating bestGatesetClosestUnitaryTable table...
Generating bestGatesetVsTargetTable table...
Generating bestGatesetErrorGenTable table...
Generating fiducialListTable table...
Generating rhoStrListTable table...
Generating EStrListTable table...
Generating germListTable table...
Generating chi2ProgressTable table...
--- Gauge Optimization to a target (L-BFGS-B) ---
The resulting Frobenius-norm distance is: 0.0387306
  frobenius norm diff of Gi = 0.172389
  frobenius norm diff of Gx = 0.17194
  frobenius norm diff of Gy = 0.17524
  frobenius norm diff of rho0 = 0.0022892
  frobenius norm 

In [6]:
import pygsti.report.latex as LU
reload(LU)
LU.latex_value(1.2e30,2)

'1\\e{30}'